In [33]:
import torch
from models import NCF

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [34]:
model = NCF(6040, 3416, 128, [128, 64, 32, 16], 1)
model.load_state_dict(torch.load('saved_models/preTrained_NCF'))

<All keys matched successfully>

In [35]:
model.eval()

NCF(
  (user_emb): Embedding(6040, 128)
  (like_emb): Embedding(3416, 128)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (relu4): ReLU()
  (outLayer): Linear(in_features=16, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [36]:
from data import TargetData

data = TargetData()

In [37]:
data.df

,uid_old,mid_old,rating,date,uid,mid,latest
0,1,1193,1,2000-12-31 22:12:40,0,0,42.0
1,1,661,1,2000-12-31 22:35:09,0,1,23.0
2,1,914,1,2000-12-31 22:32:48,0,2,28.0
3,1,3408,1,2000-12-31 22:04:35,0,3,47.0
4,1,2355,1,2001-01-06 23:38:11,0,4,4.0
...,...,...,...,...,...,...,...
999606,6040,1091,1,2000-04-26 02:35:41,6039,772,161.0
999607,6040,1094,1,2000-04-25 23:21:27,6039,1106,293.0
999608,6040,562,1,2000-04-25 23:19:06,6039,365,305.0
999609,6040,1096,1,2000-04-26 02:20:48,6039,152,234.0


In [38]:
from random import choice

row = data.df.sample(1)
row

,uid_old,mid_old,rating,date,uid,mid,latest
883119,5335,3104,1,2000-06-11 23:08:00,5334,1251,66.0


In [39]:
user = row.uid.item()

In [44]:
all_user_likes = data.df[data.df.uid==user].copy(deep=True)
not_liked = data.movies - set(all_user_likes.mid_old.values)
aul = all_user_likes.sort_values(by='mid_old').reset_index(drop=True)
aul.head()


,uid_old,mid_old,rating,date,uid,mid,latest
0,5335,21,1,2000-06-11 23:13:14,5334,145,35.0
1,5335,45,1,2000-06-11 22:58:46,5334,1276,115.0
2,5335,101,1,2000-06-11 23:02:01,5334,2409,103.0
3,5335,110,1,2000-06-11 23:12:15,5334,92,41.0
4,5335,150,1,2000-06-11 23:04:09,5334,39,91.0


In [ ]:
import numpy as np
# [x for _, x in sorted(zip(Y, X))]
# list(data.movies)
y = model(
    torch.LongTensor(np.full(data.num_movies, user)).to(device),
    torch.LongTensor(list(data.movies)).to(device)
)

In [57]:
import pandas as pd
cols = list(zip(list(data.movies),y.cpu().detach().numpy().squeeze()))

scores = pd.DataFrame(cols, columns=['mid', 'score'])

In [58]:
df = pd.merge(scores, aul, how='left')




In [60]:
df.sort_values(by='score')

,mid,score,uid_old,mid_old,rating,date,uid,latest
0,0,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
2271,2271,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
2272,2272,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
2273,2273,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
2274,2274,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...
1142,1142,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
1143,1143,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
1144,1144,0.977174,NaN,NaN,NaN,NaT,NaN,NaN
1146,1146,0.977174,5335.0,2860.0,1.0,2000-06-11 23:02:01,5334.0,101.0
